In [38]:
from dotenv import load_dotenv, find_dotenv
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

load_dotenv(find_dotenv("../config/.env"))
#openai.api_key = os.getenv("OPENAI_API_KEY")

True

In [32]:
# Document Loading 

loader = TextLoader("../data/clean-sc1015.txt")
data = loader.load()

# Document Splitting

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

# Storing into VectorDB (ChromaDB)

vectorstore = Chroma.from_documents(documents=all_splits,embedding=OpenAIEmbeddings())

In [34]:
# Build system prompt

template = """ 
You are AskNarelle, a FAQ (Frequently Asked Questions) chatbot that is designed to answer course-related queries by undergraduate students.
You are to use the provided pieces of context to answer any questions. 
If you do not know the answer, just reply with "Sorry, I'm not sure.", do not try to make up your own answer.
You are also to required to keep the answers as concise as possible.
Always end with "Thanks for using AskNarelle!" at the end of your answer.
{context}
Question: {question}
Helpful answer:
"""

In [42]:
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa = RetrievalQA.from_chain_type(llm,
                                retriever=vectorstore.as_retriever(),
                                chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

In [48]:
query = "How will the course be conducted?"
result = qa.run(query)

In [56]:
print(result)

The course will be conducted through a combination of online lectures, review lectures, physical lab sessions, and project consultations. Online lectures will be in the form of videos on NTU Learn, with quizzes. Review lectures on Data Science (DS) and Artificial Intelligence (AI) will be conducted online. Physical lab sessions will involve hands-on exercises and discussions. Project consultations will be group-wise and scheduled by appointment. Please refer to the Course Calendar and Weekly Timetable for more details. Thanks for using AskNarelle!
